# 08 - Création du dataset de données traitées pour entrainement <br> avec filtre des évènements par id
(sans encodage, ni normalisation)

Ce notebook génère 1 csv :

- dataset_for_training.csv : analyse et nettoyage des variables explicatives

Etapes : 




## A) Imports

In [79]:
import os
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
# from ydata_profiling import ProfileReport # pip install ydata_profiling ipywidgets
import matplotlib.pyplot as plt

In [80]:
# source path to datasets
path = '../data/'
data = 'dataset_for_preprocess_id_events_filtered_07.csv'
save_csv = '../data/dataset_for_training_id_events_filtered_08.csv'

In [81]:
# création d'un dataframe à partir du csv de données
df = pd.read_csv(os.path.join(path, data), header=0, parse_dates=True, index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1674 entries, 0 to 1673
Data columns (total 55 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   total_copies_job                                       1674 non-null   int64  
 1   started_at                                             1674 non-null   object 
 2   ended_at                                               1674 non-null   object 
 3   paperHeight_job                                        1674 non-null   int64  
 4   paperWidth_job                                         1674 non-null   int64  
 5   scanner_mode                                           1674 non-null   int64  
 6   bars_job                                               1674 non-null   int64  
 7   varnishConsumptionVarnish_3d_job                       1674 non-null   float64
 8   run                                             

In [82]:
df.columns

Index(['total_copies_job', 'started_at', 'ended_at', 'paperHeight_job',
       'paperWidth_job', 'scanner_mode', 'bars_job',
       'varnishConsumptionVarnish_3d_job', 'run', 'total_run',
       'copies_per_run', 'jobId', 'total_copies_requested', 'LED',
       'bars_event', 'drops', 'dithering', 'deadPixelsOffset',
       'operator_level', 'operator', 'speed_ifoil', 'ifoil', 'optifoil_ifoil',
       'stampAreas_ifoil', 'heater1Enabled_ifoil', 'speedTensionIn_ifoil',
       'speedTensionOut_ifoil', 'heater1Temperature_ifoil', 'x_imageLayout',
       'y_imageLayout', 'paperName', 'paperWidth_event', 'paperHeight_event',
       'speed', 'power_irDryers', 'power_uvDryers',
       'redScore_gridMode_remoteScannerRegistration',
       'redScore_cropmarksMode_remoteScannerRegistration',
       'x_cropmark1_cropmarksMode_remoteScannerRegistration',
       'y_cropmark1_cropmarksMode_remoteScannerRegistration',
       'x_cropmark2_cropmarksMode_remoteScannerRegistration',
       'y_cropmark2_cr

In [83]:
# Liste des colonnes à vérifier
colonnes_a_verifier = ['x_cropmark2_cropmarksMode_remoteScannerRegistration', 'speedTensionOut_ifoil', 'y_cropmark1_cropmarksMode_remoteScannerRegistration', 'enable_specialSubstrate_remoteScannerRegistration', 'x_cropmark1_cropmarksMode_remoteScannerRegistration', 'y_cropmark2_cropmarksMode_remoteScannerRegistration', 'exposureTime_manualLighting_remoteScannerRegistration']


# Afficher la répartition des valeurs pour chaque colonne dans la liste
for col in colonnes_a_verifier:
    print(f"Valeurs uniques pour {col}: {df[col].unique()}")
    print(f"Répartition des valeurs pour {col}:")
    print(df[col].value_counts())
    print("\n") 

Valeurs uniques pour x_cropmark2_cropmarksMode_remoteScannerRegistration: [   0 4477 4496 4488 3381 4517 4492 4499 4503]
Répartition des valeurs pour x_cropmark2_cropmarksMode_remoteScannerRegistration:
0       1137
4488     519
4517       6
4503       5
4492       3
4477       1
4496       1
3381       1
4499       1
Name: x_cropmark2_cropmarksMode_remoteScannerRegistration, dtype: int64


Valeurs uniques pour speedTensionOut_ifoil: [1.  0.8 0.5 0.  0.6 0.2 0.4]
Répartition des valeurs pour speedTensionOut_ifoil:
1.0    1515
0.2      76
0.0      32
0.6      29
0.5      11
0.8       8
0.4       3
Name: speedTensionOut_ifoil, dtype: int64


Valeurs uniques pour y_cropmark1_cropmarksMode_remoteScannerRegistration: [  0 164 145 149  94 170]
Répartition des valeurs pour y_cropmark1_cropmarksMode_remoteScannerRegistration:
0      1137
149     520
145       9
170       6
164       1
94        1
Name: y_cropmark1_cropmarksMode_remoteScannerRegistration, dtype: int64


Valeurs uniques pour ena

## B) Analyse des variables

In [84]:
# remise à zero des index
df.reset_index(drop=True,inplace=True)

In [85]:
# suppression des colonnes de metrics
# metrics_cols = [
#     'status', 
#     'source_events', 
#     'timestamp', 
#     'criticality_events', 
#     'name_modules', 
#     'type_modules',
#     'generation_modules', 
#     'value_counters_modules', 
#     'name_counters_modules',
#     'name_connected_operators',
#     'level_connected_operators',
#     'varnishLevelsTargetvolume',
#     'varnishLevelsTotalvolume'
#     ]
metrics_cols = [
    'timestamp', 
    'criticality_events', 
    'index',
    ]
df.drop(columns=metrics_cols, axis=0, inplace=True)

In [86]:
# suppression des colonnes en doublon entre job_events et jobs
job_events_cols = [
    'bars_event',
    'paperWidth_event', 
    'paperHeight_event',
    'varnishConsumptionVarnish_3d_event'
    ]
df.drop(columns=job_events_cols, axis=0, inplace=True)

In [87]:
# suppression des colonnes de variables catégorielles
cat_cols = [
    'operator_level', 
    'operator',
    'paperName', 
    'jobState',
    'jobId'
   ]
df.drop(columns=cat_cols, axis=0, inplace=True)

In [88]:
# suppression des colonnes datetime et ajout d'une colonne duration
df['started_at'] = pd.to_datetime(df['started_at'])
df['ended_at'] = pd.to_datetime(df['ended_at'])
# Calculer la différence de temps entre les deux colonnes
df['duration'] = (df['ended_at'] - df['started_at']).dt.total_seconds()
df.drop(columns=['started_at', 'ended_at'], axis=0, inplace=True)

In [89]:
df.head(3)

,total_copies_job,paperHeight_job,paperWidth_job,scanner_mode,bars_job,varnishConsumptionVarnish_3d_job,run,total_run,copies_per_run,total_copies_requested,LED,drops,dithering,deadPixelsOffset,speed_ifoil,ifoil,optifoil_ifoil,stampAreas_ifoil,heater1Enabled_ifoil,speedTensionIn_ifoil,speedTensionOut_ifoil,heater1Temperature_ifoil,x_imageLayout,y_imageLayout,speed,power_irDryers,power_uvDryers,redScore_gridMode_remoteScannerRegistration,redScore_cropmarksMode_remoteScannerRegistration,x_cropmark1_cropmarksMode_remoteScannerRegistration,y_cropmark1_cropmarksMode_remoteScannerRegistration,x_cropmark2_cropmarksMode_remoteScannerRegistration,y_cropmark2_cropmarksMode_remoteScannerRegistration,exposureTime_manualLighting_remoteScannerRegistration,redScore_fullScannerMode_remoteScannerRegistration,blueScore_fullScannerMode_remoteScannerRegistration,greenScore_fullScannerMode_remoteScannerRegistration,enable_specialSubstrate_remoteScannerRegistration,mode_remoteScannerRegistration,total_copies_event,identification_events,duration
0,64,483,330,3,2,9.227596,0,0,0,927,50,3,False,1,23.0,True,False,"[{'id': 1, 'end': 490, 'start': 0, 'height': 4...",True,-1.0,1.0,90,1500,-27,418,45,65,1500,1500,0,0,0,0,0,1500,20,20,False,3,64,454,233.497
1,193,483,330,3,2,27.826323,0,0,0,927,50,3,False,1,23.0,True,False,"[{'id': 1, 'end': 490, 'start': 0, 'height': 4...",True,-1.0,1.0,90,1500,-27,418,45,65,1500,1500,0,0,0,0,0,1500,20,20,False,3,193,454,723.370
2,25,483,330,3,2,3.807773,0,0,0,155,50,3,False,1,23.0,True,False,"[{'id': 1, 'end': 490, 'start': 0, 'height': 4...",True,-1.0,1.0,90,1500,-27,418,45,65,1500,1500,0,0,0,0,0,1500,20,20,False,3,25,454,284.478


### a) Profiling report

In [90]:
# profile = ProfileReport(df, title="Profiling Report")
# profile.to_notebook_iframe()

## C) Nettoyage

In [91]:
df.head()

,total_copies_job,paperHeight_job,paperWidth_job,scanner_mode,bars_job,varnishConsumptionVarnish_3d_job,run,total_run,copies_per_run,total_copies_requested,LED,drops,dithering,deadPixelsOffset,speed_ifoil,ifoil,optifoil_ifoil,stampAreas_ifoil,heater1Enabled_ifoil,speedTensionIn_ifoil,speedTensionOut_ifoil,heater1Temperature_ifoil,x_imageLayout,y_imageLayout,speed,power_irDryers,power_uvDryers,redScore_gridMode_remoteScannerRegistration,redScore_cropmarksMode_remoteScannerRegistration,x_cropmark1_cropmarksMode_remoteScannerRegistration,y_cropmark1_cropmarksMode_remoteScannerRegistration,x_cropmark2_cropmarksMode_remoteScannerRegistration,y_cropmark2_cropmarksMode_remoteScannerRegistration,exposureTime_manualLighting_remoteScannerRegistration,redScore_fullScannerMode_remoteScannerRegistration,blueScore_fullScannerMode_remoteScannerRegistration,greenScore_fullScannerMode_remoteScannerRegistration,enable_specialSubstrate_remoteScannerRegistration,mode_remoteScannerRegistration,total_copies_event,identification_events,duration
0,64,483,330,3,2,9.227596,0,0,0,927,50,3,False,1,23.0,True,False,"[{'id': 1, 'end': 490, 'start': 0, 'height': 4...",True,-1.0,1.0,90,1500,-27,418,45,65,1500,1500,0,0,0,0,0,1500,20,20,False,3,64,454,233.497
1,193,483,330,3,2,27.826323,0,0,0,927,50,3,False,1,23.0,True,False,"[{'id': 1, 'end': 490, 'start': 0, 'height': 4...",True,-1.0,1.0,90,1500,-27,418,45,65,1500,1500,0,0,0,0,0,1500,20,20,False,3,193,454,723.370
2,25,483,330,3,2,3.807773,0,0,0,155,50,3,False,1,23.0,True,False,"[{'id': 1, 'end': 490, 'start': 0, 'height': 4...",True,-1.0,1.0,90,1500,-27,418,45,65,1500,1500,0,0,0,0,0,1500,20,20,False,3,25,454,284.478
3,4,483,330,3,2,0.350165,0,0,0,28,50,3,False,1,23.0,True,False,"[{'id': 1, 'end': 490, 'start': 0, 'height': 4...",True,-1.0,1.0,90,1500,-27,418,45,65,1500,1500,0,0,0,0,0,1500,20,20,False,3,4,454,91.571
4,1,483,330,3,2,0.065590,0,0,0,204,50,3,False,1,23.0,True,False,"[{'id': 1, 'end': 490, 'start': 0, 'height': 4...",True,-1.0,1.0,90,1500,-27,418,45,65,1500,1500,0,0,0,0,0,1500,20,20,False,3,1,454,49.445


In [92]:
# Liste des colonnes à vérifier
colonnes_a_verifier = ['dithering','deadPixelsOffset','speedTensionIn_ifoil']

# Vérifier le nombre de valeurs uniques pour chaque colonne dans la liste
nombres_uniques = df[colonnes_a_verifier].nunique()

print(nombres_uniques)

dithering               2
deadPixelsOffset        2
speedTensionIn_ifoil    7
dtype: int64


In [65]:
# Liste des colonnes pour lesquelles vous voulez voir les valeurs uniques
colonnes_a_verifier = ['dithering','deadPixelsOffset','speedTensionIn_ifoil']

# Afficher les valeurs uniques pour chaque colonne dans la liste
for col in colonnes_a_verifier:
    print(f"Valeurs uniques pour {col}: {df[col].unique()}")


Valeurs uniques pour dithering: [False  True]
Valeurs uniques pour deadPixelsOffset: [1 0]
Valeurs uniques pour speedTensionIn_ifoil: [-1.   0.4  0.5  0.  -0.4 -0.2 -0.6]


In [66]:
# Liste des colonnes pour lesquelles vous voulez voir la répartition des valeurs
colonnes_a_verifier = ['dithering','deadPixelsOffset','speedTensionIn_ifoil']

# Afficher la répartition des valeurs pour chaque colonne dans la liste
for col in colonnes_a_verifier:
    print(f"Valeurs uniques pour {col}: {df[col].unique()}")
    print(f"Répartition des valeurs pour {col}:")
    print(df[col].value_counts())
    print("\n") 


Valeurs uniques pour dithering: [False  True]
Répartition des valeurs pour dithering:
False    1612
True       62
Name: dithering, dtype: int64


Valeurs uniques pour deadPixelsOffset: [1 0]
Répartition des valeurs pour deadPixelsOffset:
1    1663
0      11
Name: deadPixelsOffset, dtype: int64


Valeurs uniques pour speedTensionIn_ifoil: [-1.   0.4  0.5  0.  -0.4 -0.2 -0.6]
Répartition des valeurs pour speedTensionIn_ifoil:
-1.0    1521
-0.2      76
 0.0      30
-0.4      21
 0.5      12
 0.4      11
-0.6       3
Name: speedTensionIn_ifoil, dtype: int64




In [67]:
# supression des colonnes dont les valeurs sont constantes
# contstant_cols = ['dithering','deadPixelsOffset','speedTensionIn_ifoil', 'topMargin_remoteScannerRegistration']

constant_cols = [col for col in df.columns if df[col].nunique() == 1]
print(constant_cols)
df.drop(columns=constant_cols, axis=0, inplace=True)

['optifoil_ifoil']


In [68]:
df.head(5)

,total_copies_job,paperHeight_job,paperWidth_job,scanner_mode,bars_job,varnishConsumptionVarnish_3d_job,run,total_run,copies_per_run,total_copies_requested,LED,drops,dithering,deadPixelsOffset,speed_ifoil,ifoil,stampAreas_ifoil,heater1Enabled_ifoil,speedTensionIn_ifoil,speedTensionOut_ifoil,heater1Temperature_ifoil,x_imageLayout,y_imageLayout,speed,power_irDryers,power_uvDryers,redScore_gridMode_remoteScannerRegistration,redScore_cropmarksMode_remoteScannerRegistration,x_cropmark1_cropmarksMode_remoteScannerRegistration,y_cropmark1_cropmarksMode_remoteScannerRegistration,x_cropmark2_cropmarksMode_remoteScannerRegistration,y_cropmark2_cropmarksMode_remoteScannerRegistration,exposureTime_manualLighting_remoteScannerRegistration,redScore_fullScannerMode_remoteScannerRegistration,blueScore_fullScannerMode_remoteScannerRegistration,greenScore_fullScannerMode_remoteScannerRegistration,enable_specialSubstrate_remoteScannerRegistration,mode_remoteScannerRegistration,total_copies_event,identification_events,duration
0,64,483,330,3,2,9.227596,0,0,0,927,50,3,False,1,23.0,True,"[{'id': 1, 'end': 490, 'start': 0, 'height': 4...",True,-1.0,1.0,90,1500,-27,418,45,65,1500,1500,0,0,0,0,0,1500,20,20,False,3,64,454,233.497
1,193,483,330,3,2,27.826323,0,0,0,927,50,3,False,1,23.0,True,"[{'id': 1, 'end': 490, 'start': 0, 'height': 4...",True,-1.0,1.0,90,1500,-27,418,45,65,1500,1500,0,0,0,0,0,1500,20,20,False,3,193,454,723.370
2,25,483,330,3,2,3.807773,0,0,0,155,50,3,False,1,23.0,True,"[{'id': 1, 'end': 490, 'start': 0, 'height': 4...",True,-1.0,1.0,90,1500,-27,418,45,65,1500,1500,0,0,0,0,0,1500,20,20,False,3,25,454,284.478
3,4,483,330,3,2,0.350165,0,0,0,28,50,3,False,1,23.0,True,"[{'id': 1, 'end': 490, 'start': 0, 'height': 4...",True,-1.0,1.0,90,1500,-27,418,45,65,1500,1500,0,0,0,0,0,1500,20,20,False,3,4,454,91.571
4,1,483,330,3,2,0.065590,0,0,0,204,50,3,False,1,23.0,True,"[{'id': 1, 'end': 490, 'start': 0, 'height': 4...",True,-1.0,1.0,90,1500,-27,418,45,65,1500,1500,0,0,0,0,0,1500,20,20,False,3,1,454,49.445


In [69]:
# Vérifier si toutes les valeurs dans les deux colonnes sont identiques
same_col = (df['total_copies_job'] == df['total_copies_event']).all()
print("Les colonnes sont-elles identiques ?", same_col)


Les colonnes sont-elles identiques ? True


In [70]:
df.shape

(1674, 41)

In [71]:
# Supprimer la colonne 'total_copies_event'
df.drop(columns='total_copies_event', inplace=True)

# Renommer la colonne 'total_copies_job' en 'total_copies'
df.rename(columns={'total_copies_job': 'total_copies'}, inplace=True)


In [72]:
df.shape

(1674, 40)

Données disponibles au démarrage d'un job :
- 1  thumbnail* : chemin vignette projet
- 2  total_copies : nombre de copies imprimées
- 3  started_at : date de début
- 4  ended_at : date de fin
- 5  machineId* : identifiant machine
- 6  speed : vitesse d'impression
- 7  operator : opérateur
- 8  operator_level : niveau de l'opérateur
- 9  first_page_image_path_on_machine* : chemin de l'image
- 10 paperHeight : hauteur du substrat
- 11 paperWidth : largeur du substrat
- 12 paperName : appelation du substrat
- 13 paperThickness* : épaisseur du substrat (valeur unique à 0)
- 14 id_on_machine : identifiant unique du travail d'impression
- 15 total_copies_requested : nombre de copies demandées
- 16 job_thumbnail_id* : identifiant de vignette image
- 17 uses_ifoil : impression utilisant de la dorure
- 18 uses_iper* : impression nécéssitant l'iper (valeur unique à True)
- 19 scanner_mode : niveau de config du scanner
- 20 iper_bvar_count : compteur # TODO vérifier la fusion notebook 06
- 21 varnishConsumptionVarnish_3d : consommation de vernis en 3d
- 22 varnishConsumptionVarnish_2d* : consommation de vernis en 2d

*\*variables déjà supprimées dans le notebook 05 ou 06*

In [73]:
# colonnes à conserver dispos dans jobs
jobs_cols_to_keep = [
    'duration',
    'total_copies',
    'speed',
    'paperHeight_job', 
    'paperWidth_job', 
    'total_copies_requested',
    'ifoil',
    'scanner_mode', 
    'bars_job', 
    'varnishConsumptionVarnish_3d_job'
    ]
# colonnes à conserver dispos dans job events avec le tag start
jobevents_cols_to_keep = [
    'LED',
    'drops', 
    'speed_ifoil',
    'stampAreas_ifoil',
    'heater1Enabled_ifoil', 
    'heater1Temperature_ifoil',
    'x_imageLayout',
    'y_imageLayout', 
    'power_irDryers', 
    'power_uvDryers',
    'redScore_gridMode_remoteScannerRegistration',
    'redScore_cropmarksMode_remoteScannerRegistration',
    'redScore_fullScannerMode_remoteScannerRegistration',
    'blueScore_fullScannerMode_remoteScannerRegistration',
    'greenScore_fullScannerMode_remoteScannerRegistration',
    'mode_remoteScannerRegistration'
    ]
# colonnes à conserver pour la prédiction
metrics_cols_to_keep = ['identification_events']

# fusion des listes de colonnes à conserver
cols_to_keep = jobs_cols_to_keep + jobevents_cols_to_keep + metrics_cols_to_keep

# dataframe des variables conservées
df_to_encode = df[cols_to_keep].copy()

In [74]:
df_to_encode.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1674 entries, 0 to 1673
Data columns (total 27 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   duration                                              1674 non-null   float64
 1   total_copies                                          1674 non-null   int64  
 2   speed                                                 1674 non-null   int64  
 3   paperHeight_job                                       1674 non-null   int64  
 4   paperWidth_job                                        1674 non-null   int64  
 5   total_copies_requested                                1674 non-null   int64  
 6   ifoil                                                 1674 non-null   bool   
 7   scanner_mode                                          1674 non-null   int64  
 8   bars_job                                              1674

In [75]:
df_to_encode.drop(columns=['stampAreas_ifoil'], axis=0, inplace=True)

In [76]:
# cols = df_to_encode.columns
# for col in cols:
#     if df_to_encode[col].nunique() > 50 :
#         print(col, df_to_encode[col].nunique(), 'min', df_to_encode[col].min(), 'max', df_to_encode[col].max())
#     else :
#         print(col, df_to_encode[col].unique())

In [77]:
df_to_encode.identification_events.unique()

array([ 454,  356,  355,  354,  357,  383, 1000,  446,  387,  451,  359,
        386,  351], dtype=int64)

## D) Output

In [78]:
df_to_encode.to_csv(save_csv)